<a href="https://colab.research.google.com/github/gvanathip/Thai-Travel-Open-AI-Chatbot-with-GPT2/blob/main/bot_final_public.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers

In [ ]:
#hide
# Imports


"""
Fine-tuning the library models for language modeling on a text file (GPT, GPT-2, BERT, RoBERTa).
GPT and GPT-2 are fine-tuned using a causal language modeling (CLM) loss while BERT and RoBERTa are fine-tuned
using a masked language modeling (MLM) loss.
"""

import glob
import logging
import os
import pickle
import random
import re
import shutil
from typing import Dict, List, Tuple

import pandas as pd
import numpy as np
import torch

from sklearn.model_selection import train_test_split

from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from tqdm.notebook import tqdm, trange

from pathlib import Path

from transformers import (
    #MODEL_WITH_LM_HEAD_MAPPING,
    MODEL_FOR_CAUSAL_LM_MAPPING,
    WEIGHTS_NAME,
    AdamW,
    AutoConfig,
    AutoModelWithLMHead,
    AutoTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
    get_linear_schedule_with_warmup,

    AutoModelForCausalLM
)


try:
    from torch.utils.tensorboard import SummaryWriter
except ImportError:
    from tensorboardX import SummaryWriter

# Configs
logger = logging.getLogger(__name__)

#MODEL_CONFIG_CLASSES = list(MODEL_WITH_LM_HEAD_MAPPING.keys())
MODEL_CONFIG_CLASSES = list(MODEL_FOR_CAUSAL_LM_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)

In [ ]:
class Args():
    def __init__(self):
        #self.output_dir = 'output'
        self.output_dir = '/content/drive/MyDrive/IS-DS/thaigpt-next-125m7_7'
        self.model_type = 'gpt2'
        # self.model_name_or_path = 'microsoft/DialoGPT-small'
        # self.config_name = 'microsoft/DialoGPT-small'
        # self.tokenizer_name = 'microsoft/DialoGPT-small'

        self.model_name_or_path = 'wannaphong/thaigpt-next-125m'
        self.config_name = 'wannaphong/thaigpt-next-125m'
        self.tokenizer_name = 'wannaphong/thaigpt-next-125m'

        self.cache_dir = 'cached'
        #self.cache_dir = '/content/IS-DS/cached'
        self.block_size = 512
        self.do_train = True
        self.do_eval = True
        self.evaluate_during_training = False
        self.per_gpu_train_batch_size = 4
        self.per_gpu_eval_batch_size = 4
        self.gradient_accumulation_steps = 1
        self.learning_rate = 5e-5
        self.weight_decay = 0.0
        self.adam_epsilon = 1e-8
        self.max_grad_norm = 1.0
        self.num_train_epochs = 3
        self.max_steps = -1
        self.warmup_steps = 0
        self.logging_steps = 1000
        self.save_steps = 3500
        self.save_total_limit = None
        self.eval_all_checkpoints = False
        self.no_cuda = False
        #self.overwrite_output_dir = True
        self.overwrite_output_dir = True
        self.overwrite_cache = True
        self.should_continue = False
        self.seed = 42
        self.local_rank = -1
        self.fp16 = False
        self.fp16_opt_level = 'O1'

args = Args()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path,)
model = AutoModelForCausalLM.from_pretrained(args.output_dir)

In [ ]:
pip install pyngrok

In [ ]:
import os
import threading

from flask import Flask
from pyngrok import ngrok

In [ ]:
pip install line-bot-sdk

In [ ]:
#ngrok http 5000

In [ ]:
trip = pd.read_csv('/content/drive/MyDrive/IS-DS/trip_planners.csv')
trip.head()

In [ ]:
list(trip.head(1)['region'])[0]

In [ ]:
regions = list(set(trip.region))

In [ ]:
pr = pd.DataFrame(columns=['region','province'])

for r in regions:
  pv = []
  listToStr = ','.join([str(elem) for elem in list(trip[trip['region']==r]['plan'])])
  s = listToStr.replace('[','')
  s = s.replace(']','')
  s = s.replace("'",'')
  s = s.replace('days','')
  s = s.replace('day','')
  s = s.replace('วัน','')
  s = s.replace(',','')
  s = s.strip()
  print(r)
  for i in s.split(' '):
    if (len(i)==0)|(i.isnumeric()):
      continue
    else:
      new_row = dict()
      new_row['region'] = r
      new_row['province'] = i

      pr = pr.append(new_row, ignore_index=True)

In [ ]:
pr

In [ ]:
place = pd.read_csv('/content/drive/MyDrive/IS-DS/attraction.csv')
place.head()

In [ ]:
place = pd.merge(place,pr,on='province',how='left')

In [ ]:
place

In [ ]:
#place = place.reset_index()

In [ ]:
import ast

In [ ]:
set(trip.region)

In [ ]:
set(place.category)

In [ ]:
pip install pythainlp

In [ ]:
from pythainlp.tokenize import word_tokenize

In [ ]:
tpe = ' '.join([str(elem) for elem in list(set(place.type))])
print(tpe)

In [ ]:
from pythainlp.corpus import thai_stopwords
stopwords = list(thai_stopwords())

In [ ]:
list_word = list(set(word_tokenize(tpe)))

In [ ]:
type_words = [
 'กลางคืน',
 'กลางแจ้ง',
 'การแสดง',
 'กิจกรรม',
 'ชายหาด',
 'ชุมชน',
 'ช้อปปิ้ง',
 'ดอกไม้',
 'ดอย',
 'ตลาด',
 'ตลาดน้ำ',
 'ถ้ำ',
 'ถ้ำ',
 'ทะเล',
 'ทะเลสาบ',
 'ทุ่ง',
 'ท้องถิ่น',
 'ธรรมชาติ',
 'น้ำตก',
 'น้ำพุ',
 'ประวัติศาสตร์',
 'ผจญภัย',
 'ฝึกอบรม',
 'พระราชดำริ',
 'พระราชวัง',
 'พิพิธภัณฑ์',
 'พิพิธภัณฑ์สัตว์น้ำ',
 'ภูเขา',
 'มรดก',
 'ลำคลอง',
 'วัฒนธรรม',
 'วัด',
 'วิถีชีวิต',
 'ศาสนา',
 'ศิลปะ',
 'สงครามโลก',
 'สถานที่ศักดิ์สิทธิ์',
 'สถานปฏิบัติธรรม',
 'สถาปัตยกรรม',
 'สนามกอล์ฟ',
 'สปา',
 'สวน',
 'สวนสนุก',
 'สวนสัตว์',
 'สวนสาธารณะ',
 'สัตว์',
 'สัตว์ป่า',
 'สุขภาพ',
 'หมู่บ้าน',
 'หมู่เกาะ',
 'ห้องสมุด',
 'องุ่น',
 'อนุรักษ์',
 'อนุสรณ์สถาน',
 'อนุสาวรีย์',
 'อาร์ต',
 'อุทยานแห่งชาติ',
 'อ่าว',
 'เขื่อน',
 'เบียร์',
 'แกลเลอรี',
 'แคมป์',
 'แม่น้ำ',
 'แหล่งท่องเที่ยว',
 'โครงการหลวง',
 'โบราณ',
 'โบราณสถาน',
 'โบสถ์',
 'โรงงาน',
 'โรงละคร',
 'ไร่'
]

In [ ]:
region_words = ['เหนือ','กลาง','ใต้','อีสาน','ตะวันออก','ตะวันตก','ภาคเหนือ','ภาคกลาง','ภาคใต้','ภาคอีสาน','ภาคตะวันออก','ภาคตะวันตก']

In [ ]:
search_words = ['พัก','โรงแรม','เดินทาง','รีวิว','ที่พัก','นอนไหน','นอนที่ไหน','พักไหน','พักที่ไหน','ไปยังไง']

In [ ]:
province_words = list(set(place.province))

In [ ]:
def print_trip(df):
  rnd_trip = random.choice(list(df['plan']))
  tp = rnd_trip[1:-1]
  tp = tp.replace("'",'')
  tp = tp.replace('days','วัน')
  tp = tp.replace('day','วัน')
  tp = tp.replace(",",' +')
  print(tp)

  df = df[df['plan']==rnd_trip]

  rnd_place = random.choice(list(df['places']))
  #print(rnd_place[1:-1])

  df = df[df['places']==rnd_place]

  plc_lnk = random.choice(list(df['place_link']))
  plc_lnk = ast.literal_eval(plc_lnk)

  rg = list(df['region'])[0]
  
  pc_v = ''
  pc_k = ''
  for pc in plc_lnk:
    pc_k = pc_k+pc+','
    pc_v = pc_v+pc+'\n'+plc_lnk[pc]+'\n\n'

  text_y = pc_v+'\n\n'+tp+'\n'+rg+'\n'+pc_k
  print('print test : ',str(text_y))

  return str(text_y)

In [ ]:
trip = trip.drop_duplicates()
place = place.drop_duplicates()

In [ ]:
from pythainlp.spell import *

In [ ]:
from pythainlp import correct

In [ ]:
from pythainlp import word_vector
model_ = word_vector.get_model()

In [ ]:
from pythainlp.util import normalize

In [ ]:
bos = pd.read_csv('/content/drive/MyDrive/IS-DS/bos.csv')

In [ ]:
pip install gevent

In [ ]:
from googlesearch import search
from transformers.utils.dummy_pt_objects import TextDataset
from flask import Flask, request
from linebot.models import *
from linebot import *
import json
import csv

from gevent.pywsgi import WSGIServer

In [ ]:
chat_history = []
region_history = []
type_history = []
province_history = []


#step = 0

app = Flask(__name__)

line_bot_api = LineBotApi('xxx')
handler = WebhookHandler('xxx')

@app.route("/callback", methods=['POST'])
def callback():
    body = request.get_data(as_text=True)
    # print(body)
    req = request.get_json(silent=True, force=True)
    # req = request.get_json(force=False,cache=False)
    intent = req["queryResult"]["intent"]["displayName"]
    text = req['originalDetectIntentRequest']['payload']['data']['message']['text']
    msgType = req['originalDetectIntentRequest']['payload']['data']["message"]["type"]
    reply_token = req['originalDetectIntentRequest']['payload']['data']['replyToken']
    id = req['originalDetectIntentRequest']['payload']['data']['source']['userId']
    disname = line_bot_api.get_profile(id).display_name


    chat_history.append(text)

    print('id = ' + id)
    print('name = ' + disname)
    print('text = ' + text)
    print('intent = ' + intent)
    print('reply_token = ' + reply_token)



    reply(intent,text,reply_token,id,disname)

    return 'OK'


def reply(intent,text,reply_token,id,disname):

    # if intent == 'Default Fallback Intent':
    text_token = set(word_tokenize(text))
    found_type = text_token.intersection(set(type_words))
    found_region = text_token.intersection(set(region_words))
    found_province = text_token.intersection(set(province_words))
    found_search = text_token.intersection(set(search_words))

    if os.path.isfile('/content/drive/MyDrive/IS-DS/chat_history/'+disname+'.csv'):
      with open('/content/drive/MyDrive/IS-DS/chat_history/'+disname+'.csv','a') as f:
        writer = csv.writer(f)
        writer.writerow([disname,text])
        user_type = 'existing user'
        print('new_user:',user_type)
    # else:
    #   with open('/content/drive/MyDrive/IS-DS/chat_history/'+disname+'.csv','w') as f:
    #     writer = csv.writer(f)
    #     writer.writerow(['display_name','text']) 
    #     writer.writerow([disname,text])      
    #     user_type = 'new user'
    #     print('new_user:',user_type)




      if len(chat_history) > 1:

          if len(list(found_search)) > 0:
            y = ''
            for j in search(text, num=5, stop=5, pause=2):
              y = y+'\n'+j
            chat_history.append(y)
            with open('/content/drive/MyDrive/IS-DS/chat_history/'+disname+'.csv','a') as f:
              writer = csv.writer(f)
              writer.writerow(['bot',y])
            text_message = TextSendMessage(text=y)
            line_bot_api.reply_message(reply_token,text_message)   

          elif len(list(found_region))*len(list(found_type))*len(list(found_province)) > 0:
            print('region :',found_region)
            print('type :',found_type)
            print('province :',found_province)
            its_type = list(found_type)
            itss_type = '|'.join([str(elem) for elem in its_type])
            
            type_history.append(itss_type)

            if type_history[-1] == 'ทะเล':
              place_df = place[place['place'].str.contains('ทะเลสาบ')==False]
            else:
              place_df = place.copy()
              place_df = place_df.drop_duplicates()
            place_df = place_df[place_df['place'].str.contains(type_history[-1])==True]

            its_province = list(found_province)
            itss_province = '|'.join([str(elem) for elem in its_province])
            
            province_history.append(itss_province)

            place_df = place_df[place_df['province'].str.contains(province_history[-1])==True]

            plc_lnk = random.choices(list(place_df['link']), k=3)

            rnd_place_df = place[place['link'].isin(plc_lnk)]
            df = rnd_place_df.reset_index()  # make sure indexes pair with number of rows
            df = df.drop_duplicates()
            y = ''
            for index, row in df.iterrows():
              y = y+'\n'+row['place']+'\n'+row['link']

            chat_history.append(y)
            with open('/content/drive/MyDrive/IS-DS/chat_history/'+disname+'.csv','a') as f:
              writer = csv.writer(f)
              writer.writerow(['bot',y])          
            text_message = TextSendMessage(text=y)
            line_bot_api.reply_message(reply_token,text_message)

          elif len(list(found_type))*len(list(found_province)) > 0:
            print('type :',found_type)
            print('province :',found_province)
            its_type = list(found_type)
            itss_type = '|'.join([str(elem) for elem in its_type])
            
            type_history.append(itss_type)

            if type_history[-1] == 'ทะเล':
              place_df = place[place['place'].str.contains('ทะเลสาบ')==False]
            else:
              place_df = place.copy()
              place_df = place_df.drop_duplicates()
            place_df = place_df[place_df['place'].str.contains(type_history[-1])==True]

            its_province = list(found_province)
            itss_province = '|'.join([str(elem) for elem in its_province])
            
            province_history.append(itss_province)

            place_df = place_df[place_df['province'].str.contains(province_history[-1])==True]

            plc_lnk = random.choices(list(place_df['link']), k=3)

            rnd_place_df = place[place['link'].isin(plc_lnk)]
            df = rnd_place_df.reset_index()  # make sure indexes pair with number of rows
            df = df.drop_duplicates()
            y = ''
            for index, row in df.iterrows():
              y = y+'\n'+row['place']+'\n'+row['link']

            chat_history.append(y)
            with open('/content/drive/MyDrive/IS-DS/chat_history/'+disname+'.csv','a') as f:
              writer = csv.writer(f)
              writer.writerow(['bot',y])          
            text_message = TextSendMessage(text=y)
            line_bot_api.reply_message(reply_token,text_message)

          elif len(list(found_type))*len(list(found_region)) > 0:
            print('type :',found_type)
            print('region :',found_region)
            its_type = list(found_type)
            itss_type = '|'.join([str(elem) for elem in its_type])
            
            type_history.append(itss_type)

            if type_history[-1] == 'ทะเล':
              place_df = place[place['place'].str.contains('ทะเลสาบ')==False]
            else:
              place_df = place.copy()
              place_df = place_df.drop_duplicates()
            place_df = place_df[place_df['place'].str.contains(type_history[-1])==True]

            its_region = list(found_region)
            its_region = list(map(lambda x: x.replace('ภาค',''),its_region))
            its_region = ['ภาค' + sub for sub in its_region]
            itss_region = '|'.join([str(elem) for elem in its_region])
            itss_region = itss_region.replace('อีสาน','ตะวันออกเฉียงเหนือ')
            
            region_history.append(itss_region)

            place_df = place_df[place_df['region'].str.contains(region_history[-1])==True]

            plc_lnk = random.choices(list(place_df['link']), k=3)

            rnd_place_df = place[place['link'].isin(plc_lnk)]
            df = rnd_place_df.reset_index()  # make sure indexes pair with number of rows
            df = df.drop_duplicates()
            y = ''
            for index, row in df.iterrows():
              y = y+'\n'+row['place']+'\n'+row['link']

            chat_history.append(y)
            with open('/content/drive/MyDrive/IS-DS/chat_history/'+disname+'.csv','a') as f:
              writer = csv.writer(f)
              writer.writerow(['bot',y])          
            text_message = TextSendMessage(text=y)
            line_bot_api.reply_message(reply_token,text_message)

          elif len(list(found_region))>0:
            its = list(found_region)
            its = list(map(lambda x: x.replace('ภาค',''),its))
            its = ['ภาค' + sub for sub in its]
            itss = '|'.join([str(elem) for elem in its])
            itss = itss.replace('อีสาน','ตะวันออกเฉียงเหนือ')
            
            region_history.append(itss)

            trip_df = trip[trip['region'].str.contains(itss)==True]
            y = print_trip(trip_df)

            chat_history.append(y)
            with open('/content/drive/MyDrive/IS-DS/chat_history/'+disname+'.csv','a') as f:
              writer = csv.writer(f)
              writer.writerow(['bot',y])          
            text_message = TextSendMessage(text=y)
            line_bot_api.reply_message(reply_token,text_message)



          elif len(list(found_type))>0:
            its = list(found_type)
            itss = '|'.join([str(elem) for elem in its])
            
            type_history.append(itss)

            if type_history[-1] == 'ทะเล':
              place_df = place[place['place'].str.contains('ทะเลสาบ')==False]
            else:
              place_df = place.copy()
              place_df = place_df.drop_duplicates()
            place_df = place_df[place_df['place'].str.contains(type_history[-1])==True]

            plc_lnk = random.choices(list(place_df['link']), k=3)

            rnd_place_df = place[place['link'].isin(plc_lnk)]
            df = rnd_place_df.reset_index()  # make sure indexes pair with number of rows
            df = df.drop_duplicates()
            y = ''
            for index, row in df.iterrows():
              y = y+'\n'+row['place']+'\n'+row['link']
                # print(row['c1'], row['c2'])
            
            chat_history.append(y)
            with open('/content/drive/MyDrive/IS-DS/chat_history/'+disname+'.csv','a') as f:
              writer = csv.writer(f)
              writer.writerow(['bot',y])          
            text_message = TextSendMessage(text=y)
            line_bot_api.reply_message(reply_token,text_message)

          elif len(list(found_province))>0:
            its = list(found_province)
            itss = '|'.join([str(elem) for elem in its])
            
            province_history.append(itss)

            place_df = place.copy()
            place_df = place_df[place_df['province'].str.contains(province_history[-1])==True]

            plc_lnk = random.choices(list(place_df['link']), k=3)

            rnd_place_df = place[place['link'].isin(plc_lnk)]
            df = rnd_place_df.reset_index()  # make sure indexes pair with number of rows
            df = df.drop_duplicates()
            y = ''
            for index, row in df.iterrows():
              y = y+'\n'+row['place']+'\n'+row['link']
                # print(row['c1'], row['c2'])          

            # trip_df = trip[trip['plan'].str.contains(province_history[-1])==True]
            # y = print_trip(trip_df)

            chat_history.append(y)
            with open('/content/drive/MyDrive/IS-DS/chat_history/'+disname+'.csv','a') as f:
              writer = csv.writer(f)
              writer.writerow(['bot',y])          
            text_message = TextSendMessage(text=y)
            line_bot_api.reply_message(reply_token,text_message)   


          elif ('เที่ยว' in text) or ('ไปไหนดี') in text:
  
            y = print_trip(trip)

            # print(pics)
            chat_history.append(y)
            with open('/content/drive/MyDrive/IS-DS/chat_history/'+disname+'.csv','a') as f:
              writer = csv.writer(f)
              writer.writerow(['bot',y])          
            text_message = TextSendMessage(text=y)
            line_bot_api.reply_message(reply_token,text_message)   

  

          elif text in ['ไม่เอา','เอาใหม่']:
            y = chat_history[-2]
            with open('/content/drive/MyDrive/IS-DS/chat_history/'+disname+'.csv','a') as f:
              writer = csv.writer(f)
              writer.writerow(['bot',y])          
            text_message = TextSendMessage(text=y)
            line_bot_api.reply_message(reply_token,text_message)


    

          else:
            # if chat_history[-2] > 0:
            #   try:
            #     x = model_.most_similar_cosmul(positive=[normalize(text)])[0][0]
            #   except:
            #     x = random.choice(list(bos['0']))
            # else:
            x = normalize(text)
            
            new_user_input_ids = tokenizer.encode(x + tokenizer.eos_token, return_tensors='pt')
            #bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids
            try:
              bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1)
              #bot_input_ids = new_user_input_ids

              chat_history_ids = model.generate(
                  bot_input_ids, max_length=100,
                  pad_token_id=tokenizer.eos_token_id,
                  top_p=0.92, top_k = 50
                )
            except:
              bot_input_ids = new_user_input_ids
              chat_history_ids = model.generate(
                  bot_input_ids, max_length=100,
                  pad_token_id=tokenizer.eos_token_id,
                  top_p=0.92, top_k = 50
                )
        
            y = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
            while True:
              if chat_history[-10:].count(y)>0:
                try:
                  # x = model_.most_similar_cosmul(positive=[normalize(text)])[0][0]
                  x = model_.most_similar_cosmul(positive=[x])[0][0]
                except:
                  x = random.choice(list(bos['0']))
                new_user_input_ids = tokenizer.encode(x + tokenizer.eos_token, return_tensors='pt')
                #bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids
                try:
                  bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1)
                  #bot_input_ids = new_user_input_ids

                  chat_history_ids = model.generate(
                      bot_input_ids, max_length=300,
                      pad_token_id=tokenizer.eos_token_id,
                      top_p=0.92, top_k = 50
                    )
                except:
                  bot_input_ids = new_user_input_ids
                  chat_history_ids = model.generate(
                      bot_input_ids, max_length=300,
                      pad_token_id=tokenizer.eos_token_id,
                      top_p=0.92, top_k = 50
                    )
                y = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
              else:
                break

              

          

            y = y.replace('name',disname)
            y = y.replace('มึง',disname)
            y = y.replace('ภู ',disname)
            y = y.replace('อีเชี่ย','')
            y = y.replace('�','')
            y = y.replace('ครูลูกตาล','')
            y = y.replace('ทิม','')
            y = correct(y)

            if len(y) < 1:
              y = 'อะไรนะ เราไม่เข้าใจ'

            chat_history.append(y)
            with open('/content/drive/MyDrive/IS-DS/chat_history/'+disname+'.csv','a') as f:
              writer = csv.writer(f)
              writer.writerow(['bot',y])

            text_message = TextSendMessage(text=y)
            line_bot_api.reply_message(reply_token,text_message)


      if len(chat_history) == 1 or user_type == 'new user':
        welcome_back = ['สวัสดี','หวัดดี','ดีจ้า']
        welcome_back.append(text)
        y = random.choice(welcome_back)
        y = y+' เดี๋ยวเราช่วยหาข้อมูลที่เที่ยว ที่พัก รีวิว ให้เอง คุยกับเราได้เลย'
        chat_history.append(y)
        with open('/content/drive/MyDrive/IS-DS/chat_history/'+disname+'.csv','a') as f:
          writer = csv.writer(f)
          writer.writerow(['bot',y])      
        text_message = TextSendMessage(text=y)
        line_bot_api.reply_message(reply_token,text_message)

      print('reply = '+ y)

    else:
      with open('/content/drive/MyDrive/IS-DS/chat_history/'+disname+'.csv','w') as f:
        writer = csv.writer(f)
        writer.writerow(['display_name','text']) 
        writer.writerow([disname,text])      
        user_type = 'new user'
        print('new_user:',user_type)

        welcome_back = ['สวัสดี','หวัดดี','ดีจ้า']
        welcome_back.append(text)
        y = random.choice(welcome_back)
        y = y+' เดี๋ยวเราช่วยหาข้อมูลที่เที่ยว ที่พัก รีวิว ให้เอง คุยกับเราได้เลย'
        chat_history.append(y)
        with open('/content/drive/MyDrive/IS-DS/chat_history/'+disname+'.csv','a') as f:
          writer = csv.writer(f)
          writer.writerow(['bot',y])      
        text_message = TextSendMessage(text=y)
        line_bot_api.reply_message(reply_token,text_message)

        print('reply = '+ y)


if __name__ == '__main__':
    # Debug/Development
    # app.run(debug=True, host="0.0.0.0", port="5000")
    # Production
    http_server = WSGIServer(('', 5000), app)
    http_server.serve_forever()